# Stanford Dogs Dataset
[Stanford Dogs](http://vision.stanford.edu/aditya86/ImageNetDogs/) is a dataset of 120 dog breeds. In this project I am using YOLOv8 to train a model to detect these breeds. The dataset is divided into 120 folders, each containing images of a specific breed. 

In [ ]:
import os
import xml.etree.ElementTree as ET

CURRENT_DIR = os.getcwd()

# define paths
ROOT_DIR = f"{CURRENT_DIR}/pets"
ANNOTATIONS_DIR = os.path.join(ROOT_DIR, "Annotation")
IMAGES_DIR = os.path.join(ROOT_DIR, "Images")
YOLO_LABELS_DIR = os.path.join(ROOT_DIR, "labels")

In [114]:
# get every type of dog
def get_dog_types():
    dog_types = []
    for folder in os.listdir(ANNOTATIONS_DIR):
        if folder == ".DS_Store":
            continue
        for annotation_file in os.listdir(os.path.join(ANNOTATIONS_DIR, folder)):
            #print(annotation_file)
            tree = ET.parse(os.path.join(ANNOTATIONS_DIR,folder, annotation_file))
            root = tree.getroot()
            for obj in root.findall("object"):
                name = obj.find("name").text
                if name not in dog_types:
                    dog_types.append(name)
    return dog_types

CLASSES = get_dog_types()
print(CLASSES)
print(len(CLASSES))
def class_number(name):
    return CLASSES.index(name)

print(class_number("Chihuahua"))    


['silky_terrier', 'Scottish_deerhound', 'Chesapeake_Bay_retriever', 'Ibizan_hound', 'wire-haired_fox_terrier', 'Saluki', 'cocker_spaniel', 'schipperke', 'borzoi', 'Pembroke', 'komondor', 'Staffordshire_bullterrier', 'standard_poodle', 'Eskimo_dog', 'English_foxhound', 'golden_retriever', 'Sealyham_terrier', 'Japanese_spaniel', 'miniature_schnauzer', 'malamute', 'malinois', 'Pekinese', 'giant_schnauzer', 'Mexican_hairless', 'Doberman', 'standard_schnauzer', 'dhole', 'German_shepherd', 'Bouvier_des_Flandres', 'Siberian_husky', 'Norwich_terrier', 'Irish_terrier', 'Norfolk_terrier', 'Saint_Bernard', 'Border_terrier', 'briard', 'Tibetan_mastiff', 'bull_mastiff', 'Maltese_dog', 'Kerry_blue_terrier', 'kuvasz', 'Greater_Swiss_Mountain_dog', 'Lakeland_terrier', 'Blenheim_spaniel', 'basset', 'West_Highland_white_terrier', 'Chihuahua', 'Border_collie', 'redbone', 'Irish_wolfhound', 'bluetick', 'miniature_poodle', 'Cardigan', 'EntleBucher', 'Norwegian_elkhound', 'German_short-haired_pointer', 'Ber

In [115]:
# create yolo labels directory
if not os.path.exists(YOLO_LABELS_DIR):
    os.makedirs(YOLO_LABELS_DIR)

In [116]:
# get all of the folders in the annotations directory
folders = os.listdir(ANNOTATIONS_DIR)
for folder in folders:
    os.makedirs(os.path.join(YOLO_LABELS_DIR, folder), exist_ok=True)

In [117]:
# create all of the yolo-labels
# this just takes the first object in the xml file, and creates a yolo label for it, 
# note some xml files have multiple objects, so this is not a complete solution
def parse_xml(file):
    # parse the xml file
    tree = ET.parse(os.path.join(ANNOTATIONS_DIR, folder, file))
    root = tree.getroot()

    # get the image size
    size = root.find("size")
    width = int(size.find("width").text)
    height = int(size.find("height").text)

    # create a new file for the yolo labels
    with open(os.path.join(YOLO_LABELS_DIR, folder, file.replace(".xml", ".txt")), "w") as f:
        # go through each object in the xml file
        for obj in root.findall("object"):
            # get the class name
            class_name = obj.find("name").text

            # get the bounding box
            bndbox = obj.find("bndbox")
            xmin = int(bndbox.find("xmin").text)
            ymin = int(bndbox.find("ymin").text)
            xmax = int(bndbox.find("xmax").text)
            ymax = int(bndbox.find("ymax").text)

            # calculate the center of the bounding box
            x = (xmin + xmax) / 2 / width
            y = (ymin + ymax) / 2 / height

            # calculate the width and height of the bounding box
            w = (xmax - xmin) / width
            h = (ymax - ymin) / height

            class_name = class_number(class_name)

            return f'{class_name} {x} {y} {w} {h}\n'


# go through each folder in the annotations directory, and get all of the xml files
for folder in folders:
    if folder == ".DS_Store": continue
    for file in os.listdir(os.path.join(ANNOTATIONS_DIR, folder)):
        output = parse_xml(file)
        # place output in yolo-labels directory
        with open(os.path.join(YOLO_LABELS_DIR, folder, file + ".txt"), "w") as f:
            f.write(output)


In [118]:
# Process annotations
image_paths = []
for class_folder in os.listdir(IMAGES_DIR):
    class_folder_path = os.path.join(IMAGES_DIR, class_folder)
    
    if not os.path.isdir(class_folder_path): continue

    for image_filename in os.listdir(class_folder_path):
        #print(image_filename)
        image_paths.append(os.path.join(class_folder_path, image_filename))

print(len(image_paths))
print(image_paths[0])

20580
/Users/nickaloyd/Documents/Data Mining Final Project copy/pets/Images/n02097658-silky_terrier/n02097658_26.jpg


In [ ]:
from sklearn.model_selection import train_test_split

# split dataset into training and validation sets
train_images, val_images = train_test_split(image_paths, test_size=0.2, random_state=42)

# select only 10 images for testing, to speed up the process
train_images = train_images
val_images = val_images

# write image paths to train.txt and val.txt
with open(os.path.join(ROOT_DIR, "train.txt"), "w") as f:
    f.writelines(f"{img}\n" for img in train_images)
with open(os.path.join(ROOT_DIR, "val.txt"), "w") as f:
    f.writelines(f"{img}\n" for img in val_images)

Directory structure:
--------------------

``` text
data/
├── train/
│   ├── images/
│   │   ├── img1.jpg
│   │   ├── img2.jpg
│   ├── labels/
│       ├── img1.txt
│       ├── img2.txt
├── val/
    ├── images/
    │   ├── img3.jpg
    ├── labels/
        ├── img3.txt
```

In [120]:
# create a new folder tree for the training and validation images
DATA_DIR = os.path.join(ROOT_DIR, "data")

TRAIN_DIR = os.path.join(DATA_DIR, "train")
TRAIN_IMAGES_DIR = os.path.join(TRAIN_DIR, "images")
TRAIN_LABELS_DIR = os.path.join(TRAIN_DIR, "labels")

VAL_DIR = os.path.join(DATA_DIR, "val")
VAL_IMAGES_DIR = os.path.join(VAL_DIR, "images")
VAL_LABELS_DIR = os.path.join(VAL_DIR, "labels")

os.makedirs(TRAIN_IMAGES_DIR, exist_ok=True)
os.makedirs(TRAIN_LABELS_DIR, exist_ok=True)
os.makedirs(VAL_IMAGES_DIR, exist_ok=True)
os.makedirs(VAL_LABELS_DIR, exist_ok=True)


In [121]:
import os
import shutil

# loop through each training image
for i, image_path in enumerate(train_images):
    # derive label path
    label_path = image_path.replace(IMAGES_DIR, YOLO_LABELS_DIR).replace('.jpg', '.txt')
    
    # construct new file paths for images and labels
    new_train_image_path = f"{TRAIN_IMAGES_DIR}/{i}.jpg"
    new_train_label_path = f"{TRAIN_LABELS_DIR}/{i}.txt"

    new_val_image_path = f"{VAL_IMAGES_DIR}/{i}.jpg"
    new_val_label_path = f"{VAL_LABELS_DIR}/{i}.txt"
    
    # copy image and label to the new directories
    shutil.copy(image_path, new_train_image_path)
    shutil.copy(label_path, new_train_label_path)
    shutil.copy(image_path, new_val_image_path)
    shutil.copy(label_path, new_val_label_path)

In [122]:
# create data.yaml
import yaml
with open(os.path.join(DATA_DIR, "data.yaml"), "w") as f:
    yaml_content = {
        'train': os.path.join(ROOT_DIR, "train.txt"),
        'val': os.path.join(ROOT_DIR, "val.txt"),
        'nc': len(CLASSES),
        'names': CLASSES
    }
    yaml.dump(yaml_content, f)

print("Dataset preparation complete!")

Dataset preparation complete!


In [123]:
# create training and validation .txt files
with open(os.path.join(ROOT_DIR, "train.txt"), "w") as f:
    f.writelines(f"{TRAIN_IMAGES_DIR}/{i}.jpg\n" for i in range(len(train_images)))
with open(os.path.join(ROOT_DIR, "val.txt"), "w") as f:
    f.writelines(f"{VAL_IMAGES_DIR}/{i}.jpg\n" for i in range(len(val_images)))

yolo task=detect mode=train model=yolov8n.pt data=data/data.yaml epochs=50 imgsz=640